In [ ]:
import pystac_client
import pystac
import odc.stac
import xarray 
import rioxarray
import planetary_computer
import pathlib
import pandas
import numpy
import folium
import plotly.express
import geoviews
import geoviews.feature
import cartopy 
import dask.distributed
import branca.element, branca.colormap # Remove whitespace around small folium map
import bokeh

geoviews.extension('bokeh', 'matplotlib')

In [ ]:
def geopandas_bounds_to_plot(dataframe, crs=4326):
    """ Changing bounding box representation to leaflet notation ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))`` """
    x1, y1, x2, y2 = dataframe.to_crs(crs).total_bounds
    return ((y1, x1), (y2, x2))

In [ ]:
def update_raster_defaults(raster):
    for key in raster.data_vars:
        if key == "SCL":
            continue
        raster[key].rio.write_crs(raster[key].rio.crs, inplace=True)
        raster[key].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [ ]:
def plot_folium(data, colour_range, name, land = None):
    
    fig = branca.element.Figure(width='70%', height='50%') # Ensures no extra whitespace below
    
    m = folium.Map()
    if land is not None:
        land.explore(m=m, style_kwds=dict(color="black"),  marker_kwds=dict(fill=True))
    data.odc.add_to(m, opacity=0.75, cmap="inferno", vmin=colour_range[0], vmax=colour_range[1]) # viridis
    m.fit_bounds(data.odc.map_bounds())
    
    colormap = branca.colormap.linear.inferno.scale(colour_range[0], colour_range[1])
    colormap.caption = name
    colormap.add_to(m)
    
    display(m)

# Dask for performance

In [ ]:
client = dask.distributed.Client()
odc.stac.configure_rio(cloud_defaults=True, client=client)
display(client)

# Path setup for data within the repository

In [ ]:
data_path = pathlib.Path.cwd() / ".." / "data"
crs_wsg = 4326
crs = 2193
filter_cloud_percentage = 30
ocean_cloud_threshold = 10
name = "waikouaiti"
date_format = "%Y-%m-%d"
(data_path / "rasters" / name).mkdir(parents=True, exist_ok=True)

# STAC band names
* information on Planetary Computer Catalogue - https://stacindex.org/catalogs/microsoft-pc#/
* Information on the Copernicus DEM's - https://object.cloud.sdsc.edu/v1/AUTH_opentopography/www/metadata/Copernicus_metadata.pdf
* Good example notebook working with Copernicus Planteary Computer DEM [link](https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/copernicus-dem/copernicus-dem-example.ipynb)

In [ ]:
catalogue = {"url": "https://planetarycomputer.microsoft.com/api/stac/v1",
             "collections": {"sentinel": "sentinel-2-l2a", "dem": "cop-dem-glo-30"}}
bands = ["red", "green", "blue", "nir", "SCL", "swir16", "B05", "B8A"] # Band 05 - Vegetation red edge 1, Band 8A - Vegetation red edge 4
raster_defaults = {"resolution": 10, "nodata": 0, "dtype": "uint16"}

scl_dict = {"no data": 0, "defective": 1, "cast shadow": 2, "cloud shadow": 3,
            "vegetation": 4, "not vegetated": 5, "water": 6, "unclassified": 7,
            "cloud medium probability": 8, "cloud high probability": 9,
            "thin cirrus": 10, "snow": 11}
thresholds = {"min_ndvi": 0.01, "max_ndvi": 0.7, "max_ndwi": 0.2, "min_ndvri": 0.05}

In [ ]:
# use publically available stac link such as
odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
client = pystac_client.Client.open(catalogue["url"], modifier=planetary_computer.sign_inplace) 

# Geometry of AOI
import geopandas
geometry_df = geopandas.read_file(data_path / "vectors" / f"{name}.gpkg")
geometry = geometry_df.to_crs(crs_wsg).iloc[0].geometry
land = geopandas.read_file(data_path / "vectors" / f"main_islands.gpkg")

# Query for the data

In [ ]:
date_YYMM = "2017-02"
filters = {"eo:cloud_cover":{"lt": cloud_percentage}} # cloud_percentage
search_sentinel = client.search(
    collections=[catalogue["collections"]["sentinel"]], intersects=geometry, datetime=date_YYMM, query=filters
) 
search_dem = client.search(collections=[catalogue["collections"]["dem"]], intersects=geometry) 

pandas.DataFrame.from_records(search_sentinel.item_collection_as_dict()['features'])

## Information about the catalogues

In [ ]:
collections = list(client.get_collections())
print(f"Number of collections: {len(collections)}")
print("Collections IDs:")
for collection in collections:
    if "dem" in collection.id.lower():
        print(f"- {collection.id}")

In [ ]:
search_sentinel.item_collection()

# Download and constuct Kelp layer

In [ ]:
data = odc.stac.load(search_sentinel.items(), geopolygon=geometry, bands=bands, chunks={}, groupby="solar_day",
                     resolution = raster_defaults["resolution"], dtype=raster_defaults["dtype"], nodata=raster_defaults["nodata"],
                     patch_url=planetary_computer.sign)

## Call below if you want to load a DEM
Currently DEM is not used

In [ ]:
signed_asset = planetary_computer.sign(list(search_dem.items())[0].assets["data"])
dem = rioxarray.open_rasterio(signed_asset.href).squeeze().drop_vars("band")
dem.to_netcdf(data_path / "rasters" / {name} / "dem.nc")

## Remove any dates with no valid data

In [ ]:
data["SCL"].load()
data["SCL"] = data["SCL"].rio.clip(land.to_crs(data["SCL"].rio.crs).geometry.values, invert=True)
data["SCL"].rio.write_crs(data["SCL"].rio.crs, inplace=True);
data = data.isel(time=(data["SCL"] != scl_dict["no data"]).any(dim=["x", "y"])); # 0 == no SCL data

In [ ]:
ocean_mask = data["SCL"].isel(time=0).copy(deep=True)
ocean_mask.data[:] = 1
ocean_mask = ocean_mask.rio.clip(land.to_crs(ocean_mask.rio.crs).geometry.values, invert=True)
# Mask by time - initially sums of clud values then true / false by time if less than cloud threshold
cloud_mask = (data["SCL"] == scl_dict["cloud high probability"]).sum(dim=["x", "y"]) 
cloud_mask += (data["SCL"] == scl_dict["thin cirrus"]).sum(dim=["x", "y"])
cloud_mask += (data["SCL"] == scl_dict["defective"]).sum(dim=["x", "y"])
cloud_mask += (data["SCL"] == scl_dict["no data"]).sum(dim=["x", "y"]) - int(ocean_mask.sum())
cloud_mask = (cloud_mask / int(ocean_mask.sum())) < ocean_cloud_threshold
data = data.isel(time=(cloud_mask));

## Caclulate kelp for remaining dates
* Convert bands to floats
* Caclulate derived indices
* Mask out non-kelp areas

In [ ]:
for band in bands: 
    if band == "SCL": 
        continue
    data[band] = data[band].astype("float32").where(data[band] != 0, numpy.nan)
update_raster_defaults(data)

In [ ]:
data["ndvi"] = (data.nir - data.red) / (data.nir + data.red)
data["ndwi"] = (data.green-data.nir)/(data.green+data.nir)
data["ndvri"] = (data.B05-data.red)/(data.B05+data.red);
data["ndwi2"] = (data.swir16-data.B05)/(data.swir16+data.B05);
update_raster_defaults(data)

In [ ]:
data.to_netcdf(data_path / "rasters" / name / f'all_bands_{date_YYMM}.nc', format="NETCDF4", engine="netcdf4")

In [ ]:
#data["kelp"] = data["kelp"].where(mask, numpy.nan) <= nan set where the mask values are false
data["kelp"] = (data.nir - data.red) / (data.nir + data.red)
data["kelp"] = data["kelp"].where(data["ndvi"].data > thresholds["min_ndvi"], numpy.nan)
data["kelp"] = data["kelp"].where(data["ndvi"].data < thresholds["max_ndvi"], numpy.nan)
data["kelp"] = data["kelp"].where(data["ndwi"].data < thresholds["max_ndwi"], numpy.nan)
data["kelp"] = data["kelp"].where(data["ndvri"].data > thresholds["min_ndvri"], numpy.nan)
data["kelp"] = data["kelp"].rio.clip(land.to_crs(data["kelp"].rio.crs).geometry.values, invert=True)
data["kelp"] = data["kelp"].where(data["SCL"] != scl_dict["cloud high probability"], numpy.nan)
data["kelp"] = data["kelp"].where(data["SCL"] != scl_dict["thin cirrus"], numpy.nan)
data["kelp"] = data["kelp"].where(data["SCL"] != scl_dict["defective"], numpy.nan)
update_raster_defaults(data)

In [ ]:
data["kelp"].to_netcdf(data_path / "rasters" / name / f'kelp_{date_YYMM}.nc', format="NETCDF4", engine="netcdf4")

In [ ]:
plot_folium(data=data["SCL"].isel(time=1), colour_range=[0,10], name="SCL")

# Display
## Geoviews - time scrollbar to left

In [ ]:
kelp_display = rioxarray.rioxarray.open_rasterio(data_path / "rasters" / name / f'kelp_{date_YYMM}.nc')
gv_dataset = geoviews.Dataset(kelp_display, ['x', 'y', 'time'], 'kelp',
                              crs=cartopy.crs.epsg(data.rio.crs.to_epsg()))
gv_images = gv_dataset .to(geoviews.Image)
gv_images.opts(cmap='viridis', colorbar=True, width=600, height=500) * geoviews.feature.coastline

## Folium interactive map

In [ ]:
plot_folium(data=data["kelp"].isel(time=1), colour_range=[0,1], name="Kelp Index", land=land)

# Table of Area

In [ ]:
kelp_info = {"date": [], "area": []}
for index in range(len(kelp_display.time)):
    kelp = kelp_display.isel(time=index).load()
    kelp_info["area"].append(abs(int(kelp.notnull().sum() * kelp.x.resolution * kelp.y.resolution)))
    kelp_info["date"].append(pandas.to_datetime(data.time.data[index]).strftime(date_format))
kelp_info = pandas.DataFrame.from_dict(kelp_info, orient='columns')
#figure = plotly.express.line(kelp_info, x = 'date', y = 'area', title="Area against date")
#figure.show()

'''p = bokeh.plotting.figure()
p.line(x='date', y='area', source=bokeh.models.ColumnDataSource(kelp_info), color='green')
p.title.text = 'Kelp Area by Date'
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'm^2 of Kelp'

hover = bokeh.models.tools.HoverTool()
hover.tooltips=[('Date', '@date'), ('Area', '@area'),]
p.add_tools(hover)

bokeh.plotting.show(p)'''
kelp_info